In [4]:
import pandas as pd
from difflib import get_close_matches
from datetime import datetime

# Sample dataset
df = pd.read_csv('/content/drive/MyDrive/Google_Hackathon/updated_dataset.csv')


def closest_match(symptom, symptoms_list):
    closest = get_close_matches(symptom, symptoms_list, n=1, cutoff=0.8)  # Adjust the cutoff threshold as needed
    return closest[0] if closest else None

def parse_schedule(schedule_str):
    # Parse the schedule string to extract start and end times
    start, end = schedule_str.split('-')
    start_time = datetime.strptime(start.strip(), "%I:%M %p")
    end_time = datetime.strptime(end.strip(), "%I:%M %p")
    return start_time, end_time

def recommend(symptom_1, symptom_2, symptom_3):
    if symptom_1 == symptom_2 or symptom_1 == symptom_3 or symptom_2 == symptom_3:
        return "Symptoms 1, 2, and 3 should be different and unique."

    all_symptoms = set([symptom_1, symptom_2, symptom_3])
    all_recommendations = {}

    for symptom in all_symptoms:
        closest_symptom = closest_match(symptom, df[['Symptom_1', 'Symptom_2', 'Symptom_3']].values.flatten().tolist())

        if closest_symptom:
            filtered_df = df[df.apply(lambda x: closest_symptom in x.values, axis=1)]

            if filtered_df.empty:
                continue

            for idx, row in filtered_df.iterrows():
                disease = row['Disease']
                recommendation = {
                    "Name": row['Name'],
                    "City": row['City'],
                    "State": row['State'],
                    "Rating": int(row['user_rating']),  # Convert rating to integer
                    "Schedule": parse_schedule(row['schedule'])
                }
                if disease in all_recommendations:
                    all_recommendations[disease].append(recommendation)
                else:
                    all_recommendations[disease] = [recommendation]

    # If no recommendations are found based on any symptom, return an appropriate message
    if not all_recommendations:
        return "No recommendation found"
    # Sort recommendations based on rating and schedule availability for each disease
    for disease, details in all_recommendations.items():
        # Sort by rating in descending order
        details.sort(key=lambda x: x.get('Rating', 0), reverse=True)
        # Then sort by schedule availability
        details.sort(key=lambda x: x.get('Schedule')[0])

    return all_recommendations


# Input symptoms
symptom_1 = input("Enter symptom 1: ")
symptom_2 = input("Enter symptom 2: ")
symptom_3 = input("Enter symptom 3: ")

# Get recommendations
ecommendations = recommend(symptom_1, symptom_2, symptom_3)
print("Recommendations:")
if isinstance(recommendations, str):
    print(recommendations)
else:
    for disease, doctors in recommendations.items():
        print("For Disease:", disease)
        if doctors:
            top_doctor = doctors[0]  # Select the top-rated doctor
            print("Recommendation")
            print("Name:", top_doctor["Name"])
            print("City:", top_doctor["City"])
            print("State:", top_doctor["State"])
            print("Rating:", top_doctor["Rating"])
            print("Schedule:", top_doctor["Schedule"][0].strftime("%I:%M %p"), "-", top_doctor["Schedule"][1].strftime("%I:%M %p"))
        else:
            print("No recommendation found for this disease")
        print()


Enter symptom 1: itching
Enter symptom 2: skin_rash
Enter symptom 3: nodal_skin_eruptions
Recommendations:
For Disease: Fungal infection
Recommendation
Name: DR. KOTHARI ARUNA
City: CHHINDWARA
State: MADHYA PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

For Disease: Acne
Recommendation
Name: DR. AGRAWAL UMESH
City: RATLAM
State: MADHYA PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

For Disease: Psoriasis
Recommendation
Name: DR. PANDEY S.S.
City: VARANASI
State: UTTAR PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM

For Disease: Impetigo
Recommendation
Name: DR. GUPTA CHANDRAMOHAN
City: BHOPAL
State: MADHYA PRADESH
Rating: 5
Schedule: 09:00 AM - 09:30 AM



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
